In [36]:
APIKEY="ENTER API KEY HERE"
print APIKEY


PROJECT_ID = "ENTER-PROJECT-ID-HERE"
print PROJECT_ID 

ENTER API KEY HERE
ENTER-PROJECT-ID-HERE


In [26]:
from googleapiclient.discovery import build

In [27]:
sservice = build('speech', 'v1beta1', developerKey=APIKEY)
response = sservice.speech().syncrecognize(
    body={
        'config': {
            'encoding': 'LINEAR16',
            'sampleRate': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()
print response
print response['results'][0]['alternatives'][0]['transcript']

{u'results': [{u'alternatives': [{u'confidence': 0.98267895, u'transcript': u'how old is the Brooklyn Bridge'}]}]}
how old is the Brooklyn Bridge


In [28]:

# Running Vision API
import base64
IMAGE="gs://cpb103-public-files/noirbree.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 100,
                }]
            }],
        })
outputs = request.execute(num_retries=3)
#print outputs

for output in outputs['responses'][0]['labelAnnotations']:
  print u"{0}".format(output['description'])

dog
mammal
vertebrate
dog breed group
dog like mammal
puppy
dog crossbreeds


In [29]:
def executeTranslate(inputs):
  from googleapiclient.discovery import build
  service = build('translate', 'v2', developerKey=APIKEY)
  translator = service.translations()
  outputs = translator.list(source='en', target='es', q=inputs).execute()  
  return outputs['translations'][0]['translatedText']

print "Added executeTranslate() function."

Added executeTranslate() function.


In [30]:
alice = sc.textFile("gs://cpb103-public-files/alice-short-transformed.txt")
alice = alice.map(lambda x: x.split("."))
                  
for eachSentence in alice.take(10):
  print u"{0}".format(eachSentence)

[u'Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do', u'']
[u' Once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice, "without pictures or conversations?"  So she was considering in her own mind (as well as she could, for the day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her', u'']
[u'  There was nothing so very remarkable in that, nor did Alice think it so very much out of the way to hear the Rabbit say to itself, "Oh dear! Oh dear! I shall be too late!" But when the Rabbit actually took a watch out of its waistcoat-pocket and looked at it and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with ei

In [31]:
aliceTranslated = alice.map(executeTranslate)

for eachSentance in aliceTranslated.take(10):
    print u"{0}".format(eachSentance)

Alice estaba empezando a cansarse de estar sentada por su hermana en la orilla, y de no tener nada que hacer
Una o dos veces se había asomado al libro que su hermana estaba leyendo, pero no tenía dibujos ni diálogos en ella, &quot;y lo que es el uso de un libro&quot;, pensó Alicia, &quot;sin dibujos ni diálogos?&quot; Así que estaba pensando en su propia mente (así como pudo, para el día hacía sentirse muy somnoliento y estúpido), si el placer de hacer una conexión en cadena valdría la pena el esfuerzo de levantarse y coger las margaritas, cuando de repente, un conejo blanco con ojos rosas pasó corriendo junto a ella
No había nada tan muy notable, ya que, ni le pareció a Alicia por lo mucho fuera del camino para escuchar el conejo se decía a sí mismo, &quot;Oh cielos! ¡Ay! Voy a ser demasiado tarde!&quot; Pero cuando el conejo sacó un reloj fuera de su bolsillo del chaleco y lo miró y luego apretó el paso, Alice empezó a ponerse en pie, ya que cruzó por su mente que ella nunca antes ha

In [32]:
def executeSentimentAnalysis(quote):
    from googleapiclient.discovery import build
    lservice = build('language', 'v1beta1', developerKey=APIKEY)

    response = lservice.documents().analyzeSentiment(
        body={
            'document': {
                'type': 'PLAIN_TEXT',
                'content': quote
            }
        }).execute()
    
    return response

print "Added executeSentimentAnalysis() function."

Added executeSentimentAnalysis() function.


In [33]:
import pandas as pd
from pandas.io import gbq

print "Imports run."

Imports run.


In [34]:
print 'Running query...'
df = gbq.read_gbq('SELECT title, text FROM [bigquery-public-data:hacker_news.stories] where text > " " and title contains("JavaScript") LIMIT 10', project_id=PROJECT_ID)

#Convert Pandas DataFrame to RDD
rdd = sqlContext.createDataFrame(df).rdd

print rdd.take(2)

Running query...
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 10 rows.

Total time taken 0.83 s.
Finished at 2017-02-12 22:54:45.
[Row(title=u'Lead JavaScript for innovative intelligent learning platform at ExpertKnowledge', text=u'About Us\nAt ExpertKnowledge, we provide automation platform to extract wisdom and transform it into high-quality online learning courses that are certified and delivered on any device.\nOur team is looking to add an experienced Front End Engineer to lead the design of our web platform - both mobile and desktop.\nYou\u2019ll collaborate with our UX designers to build great user experiences. You will work closely with talented engineers to build a scalable, resilient, and well-designed applications. You\u2019ll own, architect, and expand key pieces of our platform, mentor junior engineers, and have a huge impact on the product from conception to launch.<p>You will:\nLead the design and development of next generati

In [35]:
# extract text field from Dictionary
comments = rdd.map(lambda x: x[1])
sentiments = comments.map(executeSentimentAnalysis)

for sentiment in sentiments.collect():
    print u"Score:{0} and Magnitde:{1}".format(sentiment['documentSentiment']['score'], sentiment['documentSentiment']['magnitude'])

Score:0.5 and Magnitde:3.2
Score:0.2 and Magnitde:0.2
Score:0.2 and Magnitde:0.5
Score:0.2 and Magnitde:2.1
Score:-0.7 and Magnitde:0.7
Score:0.3 and Magnitde:0.6
Score:0.2 and Magnitde:1.5
Score:-0.5 and Magnitde:0.5
Score:0.1 and Magnitde:1
Score:0.1 and Magnitde:1.9
